In [ ]:
import csv
import pandas as pd
import os
import numpy as np
#Random forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#Xgboost
from xgboost import XGBClassifier  #pip install xgboost
#svm
from sklearn import svm
#gbm
from sklearn import ensemble
#logistics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix  
from tabulate import tabulate
import time

In [ ]:
#import data
df=pd.DataFrame.from_csv("D3_features.csv")#this file should be putted in the current working directory
labels = np.array(df['V25'])
# Remove the labels from the features
# axis 1 refers to the columns
features= df.drop('V25', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

## Random Forest

In [ ]:
##traning##
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_jobs=2, random_state=0)
# Train the model on training data
rf_model = rf.fit(train_features, train_labels)
rf.score(train_features, train_labels)

In [ ]:
##testing##
# Use the forest's predict method on the test data
predictions_rf = rf.predict(test_features)
##evaluation##
rf.score(test_features,test_labels)
#confusion matrix
print(confusion_matrix(test_labels,predictions_rf))  
print(classification_report(test_labels,predictions_rf))  

## Support vector machine

In [ ]:
##training##
t10=time.time() #Track time
clf = svm.SVC(C=0.8, kernel='rbf', gamma=20, decision_function_shape='ovr')
clf.fit(train_features, train_labels)
t11=time.time()
print("Time:", t10,t11)
print("Train:",clf.score(train_features, train_labels)) # train score

In [ ]:
##testing##
predictions_svm = clf.predict(test_features)
##evaluation##
clf.score(test_features,test_labels) 
##confusion matrix##
print(confusion_matrix(test_labels,predictions_svm))  
print(classification_report(test_labels,predictions_svm))  

## Gradient Boosting machine

In [ ]:
##training##
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gbm = ensemble.GradientBoostingRegressor(**params)
gbm.fit(train_features, train_labels)
gbm.score(train_features, train_labels)  # train score

In [ ]:
###testing##
predictions_gbm = gbm.predict(test_features)
##evaluation##
errors_gbm = abs(test_labels-predictions_gbm)
gbm.score(test_features,test_labels)
##confusion matrix##
predictions_gbm1=[]
for i in range(len(predictions_gbm)):
    if predictions_gbm[i]>0.5:
        predictions_gbm1.append(1)
    else:
        predictions_gbm1.append(0)
print(confusion_matrix(test_labels,predictions_gbm1))  
print(classification_report(test_labels,predictions_gbm1))  

## Xgboost

In [ ]:
##training##
xg = XGBClassifier()
xg_model=xg.fit(train_features, train_labels)
xg.score(train_features, train_labels)

In [ ]:
##testing##
predictions_xg = xg.predict(test_features)
##evaluation##
errors_xg = abs(test_labels-predictions_xg)
xg.score(test_features,test_labels)
#confusion matrix##
print(confusion_matrix(test_labels,predictions_xg))  
print(classification_report(test_labels,predictions_xg))  

In [ ]:
##features selection##
#The code for feature selection is similar for Random Forest and Xgboost
features= df.drop('V25', axis = 1)
features.columns
headers = ["name", "score"]
values2 = sorted(zip(features.columns, xg_model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values2, headers, tablefmt="plain"))

## Logistics Regression

In [ ]:
##train##
lr = LogisticRegression()
lr.fit(train_features, train_labels)
lr.score(train_features, train_labels)

In [ ]:
##test##
predictions_lr=lr.predict(test_features)
##evaluation##
error_lr=abs(test_labels-predictions_lr)
lr.score(test_features, test_labels)
##confusion matrix##
print(confusion_matrix(test_labels,predictions_lr))  
print(classification_report(test_labels,predictions_lr))  